# Data Wrangling: Mobility and COVID-19 Infection Rate

In [50]:
import pandas as pd
import numpy 
import os

In [51]:
apple_df= pd.read_csv('Data/applemobilitytrends-2020-05-05.csv')
google_df= pd.read_csv('Data/Global_Mobility_Report.csv')
cases_df= pd.read_csv('Data/us-counties.csv')

/Users/jeremysilva/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Our data of interest is held in three different datasets:

1) Apple Mobility Dataset

2) Google Mobility Dataset

3) COVID-19 Infection Dataset

### Exploring Consistency Across Datasets

In [52]:
# We need to extract only US cities from the Google data
# First we extract only rows with US data
google_df= google_df[google_df['country_region']== 'United States']
google_df.info()
google_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205643 entries, 92520 to 298162
Data columns (total 11 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   country_region_code                                 205643 non-null  object 
 1   country_region                                      205643 non-null  object 
 2   sub_region_1                                        205567 non-null  object 
 3   sub_region_2                                        201691 non-null  object 
 4   date                                                205643 non-null  object 
 5   retail_and_recreation_percent_change_from_baseline  159853 non-null  float64
 6   grocery_and_pharmacy_percent_change_from_baseline   153836 non-null  float64
 7   parks_percent_change_from_baseline                  54098 non-null   float64
 8   transit_stations_percent_change_from_baseline       79905 no

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
92520,US,United States,NaN,NaN,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
92521,US,United States,NaN,NaN,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
92522,US,United States,NaN,NaN,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
92523,US,United States,NaN,NaN,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
92524,US,United States,NaN,NaN,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0


In [53]:
# Sub_region_2 is the column that breaks the data down to a county level
# We want a dataframe that holds only the county data
google_df= google_df[google_df['sub_region_2'].notna()]
google_df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
92672,US,United States,Alabama,Autauga County,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN
92673,US,United States,Alabama,Autauga County,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN
92674,US,United States,Alabama,Autauga County,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
92675,US,United States,Alabama,Autauga County,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
92676,US,United States,Alabama,Autauga County,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0


In [54]:
# Variable that holds the number of unique U.S counties in the Google df
num_google_counties= len(google_df.sub_region_2.unique())
# List of the unique U.S. counties in the Google df
google_counties= list(google_df.sub_region_2.unique())
print('There are {} unique U.S. counties in the Google Dataset'.format(num_google_counties))

There are 1711 unique U.S. counties in the Google Dataset


In [55]:
cases_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [56]:
# Varaible that holds number of Unique U.S. counties in the Cases df
num_cases_counties=len(cases_df.county.unique())
# List of all the unqiue U.S. counties in the Cases df
cases_counties= list(cases_df.county.unique())
print('There are {} unique U.S. counties in the Cases Dataset'.format(num_cases_counties))

There are 1715 unique U.S. counties in the Cases Dataset


In [57]:
# Inspect our list of Google counties
google_counties[:10]

['Autauga County',
 'Baldwin County',
 'Barbour County',
 'Bibb County',
 'Blount County',
 'Bullock County',
 'Butler County',
 'Calhoun County',
 'Chambers County',
 'Cherokee County']

In [58]:
# Inspect our list of Case counties
cases_counties[:10]

['Snohomish',
 'Cook',
 'Orange',
 'Maricopa',
 'Los Angeles',
 'Santa Clara',
 'Suffolk',
 'San Francisco',
 'Dane',
 'San Diego']

In [59]:
# We see that in the Google df we have County affixed the the end of each county name
# To find matches it will be easier if we remove the 'County'
# We create a new empty list
google_counties_clean=[]
# We loop through google_counties, find and replace 'County', strip away white space, and then append them to new list
for county in google_counties: 
    county= county.replace('County','').strip()
    google_counties_clean.append(county)
# Now we can see the structure is the same between both Google and Cases
# Check to make sure no counties were lost
len(google_counties_clean) == len(google_counties)


True

In [60]:
# We want to see how many matches we have between counties in the Google df and counties in the cases df
google_cases_matches=[]
for i in cases_counties:
    if i in google_counties_clean:
        google_cases_matches.append(i)
print('There are {} counties that exist in both the Google and Cases datasets'.format(len(google_cases_matches)))

There are 1544 counties that exist in both the Google and Cases datasets


In [61]:
# We need to extract out only US cities from the Apple Data
# First we create a df with only cities
apple_city_df= apple_df[apple_df['geo_type']=='city']
# Find all the cities in the df
city_names= apple_city_df['region'].unique()
# Create a list that holds only us cities
us_city_names_apple= ['Albuquerque','Ann Arbor', 'Annapolis','Atlanta', 'Auckland', 'Austin', 'Bakersfield', 
        'Baltimore','Birmingham - Alabama','Boston','Charlotte', 'Chicago', 'Cincinnati',
       'Cleveland','Colorado Springs', 'Columbus','Dallas','Denver','Detroit','Fresno','Grand Rapids', 'Greensboro',
       'Honolulu', 'Houston', 'Indianapolis','Jacksonville', 'Johannesburg', 'Kansas City','Las Vegas','Los Angeles', 
        'Louisville', 'Madison', 'Madrid','Memphis','Miami','Milwaukee', 'Minneapolis', 'Modesto','Monterey', 
        'Nashville', 'New Haven', 'New Orleans', 'New York City','Oklahoma City', 'Omaha', 'Orlando','Philadelphia', 
        'Phoenix', 'Pittsburgh','Portland','Providence', 'Raleigh', 'Richmond','Rochester','Rotterdam',
       'Sacramento', 'Saint Petersburg - Clearwater (Florida)','Salt Lake City', 'San Antonio',
       'San Diego', 'San Francisco - Bay Area', 'Santa Barbara','Seattle','Spokane','St. Louis', 'Tacoma', 'Tampa',
       'Toledo','Tucson', 'Tulsa', 'Ventura','Virginia Beach','Washington DC']
# create a new df that hold the data on only US cities
apple_us_city_df= apple_city_df[apple_city_df['region'].isin(us_city_names_apple)]
us_city_names_apple= list(apple_us_city_df.region.unique())
# Recheck the data and make sure it starts with Albuqerque 
apple_us_city_df.head()

,geo_type,region,transportation_type,alternative_name,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,...,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05
153,city,Albuquerque,driving,NaN,100.0,101.06,104.03,97.01,118.28,116.32,...,57.21,69.77,73.44,73.93,76.91,90.40,80.03,63.08,77.37,81.24
154,city,Albuquerque,transit,NaN,100.0,82.47,86.27,77.21,79.91,80.35,...,28.78,34.19,36.82,35.57,40.91,44.49,37.11,35.28,37.69,38.86
155,city,Albuquerque,walking,NaN,100.0,102.02,107.65,86.33,115.66,129.90,...,65.87,75.28,83.88,87.56,89.58,101.59,92.54,69.08,86.75,94.95
159,city,Ann Arbor,driving,NaN,100.0,100.28,101.92,104.53,129.03,81.12,...,40.62,48.82,51.54,46.44,48.52,60.69,57.56,49.52,53.14,53.37
160,city,Ann Arbor,transit,NaN,100.0,103.06,94.19,95.90,98.55,64.89,...,10.44,11.55,11.48,9.50,13.05,16.51,12.52,16.40,16.21,12.56


In [62]:
print('There are {} unique U.S. cities in the Apple dataset.'.format(len(us_city_names_apple)))

There are 73 unique U.S. cities in the Apple dataset.


In [63]:
# To check for matches we need to clean up this list a bit
us_city_names_apple[us_city_names_apple.index('Birmingham - Alabama')]= 'Birmingham'
us_city_names_apple[us_city_names_apple.index('Saint Petersburg - Clearwater (Florida)')]= 'Saint Petersburg'
us_city_names_apple[us_city_names_apple.index('San Francisco - Bay Area')]= 'San Francisco'

full_match=[]
for place in us_city_names_apple:
    if place in google_cases_matches:
        full_match.append(place)
        
print('There are {} regions that match across all three datasets'.format(len(full_match)))

There are 28 regions that match across all three datasets


In [64]:
full_match

['Austin',
 'Baltimore',
 'Charlotte',
 'Cleveland',
 'Columbus',
 'Dallas',
 'Denver',
 'Fresno',
 'Honolulu',
 'Houston',
 'Los Angeles',
 'Madison',
 'Miami',
 'Milwaukee',
 'Monterey',
 'New Haven',
 'Philadelphia',
 'Providence',
 'Raleigh',
 'Richmond',
 'Sacramento',
 'San Diego',
 'San Francisco',
 'Santa Barbara',
 'Spokane',
 'St. Louis',
 'Tulsa',
 'Ventura']

### Combining Datasets

We have much more consistencty between the Google dataset and the Cases dataset than we do amongst all three.

Because of this we are going to create two datasets: one that joins just Google and Cases and another that joins all three

#### Joining Google and Cases datasets

In [65]:
google_df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
92672,US,United States,Alabama,Autauga County,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN
92673,US,United States,Alabama,Autauga County,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN
92674,US,United States,Alabama,Autauga County,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
92675,US,United States,Alabama,Autauga County,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
92676,US,United States,Alabama,Autauga County,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0


In [66]:
# We need to clean up the Google dataset to make it better match the cases dataset
# We can drop non relevant geographical columns. All we need is county name
google_df_clean= google_df.drop(['country_region_code', 'country_region','sub_region_1'], axis=1)
# Reset the index
google_df_clean.reset_index(inplace=True, drop=True)
# Rename sub_region_2 to County
google_df_clean= google_df_clean.rename(columns={'sub_region_2': 'county'})
#Recheck the new dataframe
google_df_clean.head()

,county,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,Autauga County,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN
1,Autauga County,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN
2,Autauga County,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
3,Autauga County,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
4,Autauga County,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0


In [67]:
# We want to drop county from the end of each county name so it is easier to match on
google_df_clean['county'] = google_df_clean['county'].str.replace("County", "").str.strip()
google_df_clean['county'].nunique()

1695

In [68]:
cases_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [69]:
# We can drop the state column and the fips column which is just a numerical county identifier
cases_df_clean= cases_df.drop(columns=['state', 'fips'])
# We want a slice of the dataset that starts on the same date as the Google dataset
cases_df_clean= cases_df_clean.loc[cases_df_clean['date'] >= '2020-02-15']
# Ensure that we didn't lose any counties but subsecting the data 
cases_df.county.nunique()

1715

In [70]:
# Merge the Google dataset and the Cases dataset
google_cases_df= pd.merge(google_df_clean, cases_df_clean, how='left', left_on=['county','date'], right_on = ['county','date'])
google_cases_df.head()

,county,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,cases,deaths
0,Autauga,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN,NaN,NaN
1,Autauga,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN,NaN,NaN
2,Autauga,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0,NaN,NaN
3,Autauga,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0,NaN,NaN
4,Autauga,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0,NaN,NaN


In [71]:
google_cases_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 537871 entries, 0 to 537870
Data columns (total 10 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   county                                              537871 non-null  object 
 1   date                                                537871 non-null  object 
 2   retail_and_recreation_percent_change_from_baseline  365550 non-null  float64
 3   grocery_and_pharmacy_percent_change_from_baseline   345439 non-null  float64
 4   parks_percent_change_from_baseline                  120198 non-null  float64
 5   transit_stations_percent_change_from_baseline       197110 non-null  float64
 6   workplaces_percent_change_from_baseline             520613 non-null  float64
 7   residential_percent_change_from_baseline            224518 non-null  float64
 8   cases                                               444223 non-n

#### Joining all three datasets

In [72]:
apple_us_city_df.head()

,geo_type,region,transportation_type,alternative_name,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,...,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05
153,city,Albuquerque,driving,NaN,100.0,101.06,104.03,97.01,118.28,116.32,...,57.21,69.77,73.44,73.93,76.91,90.40,80.03,63.08,77.37,81.24
154,city,Albuquerque,transit,NaN,100.0,82.47,86.27,77.21,79.91,80.35,...,28.78,34.19,36.82,35.57,40.91,44.49,37.11,35.28,37.69,38.86
155,city,Albuquerque,walking,NaN,100.0,102.02,107.65,86.33,115.66,129.90,...,65.87,75.28,83.88,87.56,89.58,101.59,92.54,69.08,86.75,94.95
159,city,Ann Arbor,driving,NaN,100.0,100.28,101.92,104.53,129.03,81.12,...,40.62,48.82,51.54,46.44,48.52,60.69,57.56,49.52,53.14,53.37
160,city,Ann Arbor,transit,NaN,100.0,103.06,94.19,95.90,98.55,64.89,...,10.44,11.55,11.48,9.50,13.05,16.51,12.52,16.40,16.21,12.56


In [73]:
# We need to clean and reshape the apple df to match the shape of the other two google and cases dataframes 
# We can drop geo_type and alternative_name
apple_us_city_df= apple_us_city_df.drop(columns=['geo_type', 'alternative_name'])
# Melt the data frame down
apple_us_city_df= apple_us_city_df.melt(id_vars= ['region', 'transportation_type'])
# Create a pivot table of the melted dataframe to extract out the transportation types into seperate columns
apple_us_city_df_pivot= apple_us_city_df.pivot_table(index=['region', 'variable'], columns='transportation_type')
# Assign the pivot to the original df with a reset index
apple_us_city_df= apple_us_city_df_pivot.reset_index()
# Rename the columns appropriately
apple_us_city_df.columns= ['county','date', 'driving','transit','walking']
# Recheck the dataframe 
apple_us_city_df.head()

,county,date,driving,transit,walking
0,Albuquerque,2020-01-13,100.00,100.00,100.00
1,Albuquerque,2020-01-14,101.06,82.47,102.02
2,Albuquerque,2020-01-15,104.03,86.27,107.65
3,Albuquerque,2020-01-16,97.01,77.21,86.33
4,Albuquerque,2020-01-17,118.28,79.91,115.66


In [74]:
# Now we need to merge the datasets
# Merge the Google dataset and the Cases dataset
google_apple_cases_df= pd.merge(google_cases_df, apple_us_city_df, left_on=['county', 'date'], right_on=['county','date'])
google_apple_cases_df.head()

,county,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,cases,deaths,driving,transit,walking
0,Dallas,2020-02-15,-1.0,-2.0,NaN,NaN,2.0,NaN,NaN,NaN,126.79,102.23,176.33
1,Dallas,2020-02-15,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.79,102.23,176.33
2,Dallas,2020-02-15,10.0,8.0,NaN,NaN,3.0,-3.0,NaN,NaN,126.79,102.23,176.33
3,Dallas,2020-02-15,8.0,-7.0,NaN,NaN,10.0,NaN,NaN,NaN,126.79,102.23,176.33
4,Dallas,2020-02-15,4.0,1.0,16.0,0.0,1.0,-1.0,NaN,NaN,126.79,102.23,176.33


In [75]:
google_apple_cases_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19200 entries, 0 to 19199
Data columns (total 13 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   county                                              19200 non-null  object 
 1   date                                                19200 non-null  object 
 2   retail_and_recreation_percent_change_from_baseline  13944 non-null  float64
 3   grocery_and_pharmacy_percent_change_from_baseline   13432 non-null  float64
 4   parks_percent_change_from_baseline                  4949 non-null   float64
 5   transit_stations_percent_change_from_baseline       9045 non-null   float64
 6   workplaces_percent_change_from_baseline             18591 non-null  float64
 7   residential_percent_change_from_baseline            10067 non-null  float64
 8   cases                                               17363 non-null  float64


In [76]:
print('In the fully matched dataframe we only have data on {} counties.'.format(google_apple_cases_df.county.nunique()))
print('In the Google and Cases dataframe we have data on {} counties'.format(google_cases_df.county.nunique()))

In the fully matched dataframe we only have data on 28 counties.
In the Google and Cases dataframe we have data on 1695 counties


In [77]:
# Export both datasets the csv
google_cases_df.to_csv('Data/google_cases_data.csv')
google_apple_cases_df.to_csv('Data/google_apple_cases_data.csv')